In [2]:
import pandas as pd
import numpy as np
import ast

import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from PIL import Image
import os

Data Cleaning

In [7]:
labels = pd.read_csv('data/small_data/labels.csv', index_col=0)
print(labels.shape)
labels.head()

(2218, 7)


,index,genes,sex,origin,price,birth,url
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...


In [8]:
# 'Normal' if there is no genes
labels.loc[labels["genes"] == "[]", "genes"] = '["Normal"]'

In [9]:
clean_genes = []
list_genes = [ast.literal_eval(gene) for gene in labels['genes']]
for lst in list_genes:
    for element in lst:
        if "Het" in element:
            clean_genes.append('Het' + ' '.join(element.split('Het')[1:]))
        else:
            clean_genes.append(element)

clean_possible_genes = list(set(clean_genes))
print(f'Number of possible genes: {len(clean_possible_genes)}')
clean_possible_genes[:5]

Number of possible genes: 147


['Spotnose', 'Sugar', 'KRG', 'Leopard', 'Blade']

In [10]:
gene_extension_df = pd.DataFrame(np.zeros([labels.shape[0], len(clean_possible_genes)]), dtype=int, columns=clean_possible_genes)
labels_extended = pd.concat([labels, gene_extension_df], axis=1)
labels_extended.head()

,index,genes,sex,origin,price,birth,url,Spotnose,Sugar,KRG,...,Bamboo,Bongo,Desert Ghost,Het Citrus Hypo,Axanthic (TSK),Hypo,Axanthic (VPI),Orange Ghost,Stranger,Het Puzzle
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
len([len(lst) for lst in list_genes])

2218

In [12]:
start_row = 0
count = 0
for gene_col in clean_genes:
    labels_extended.loc[start_row, gene_col] = 1
    count += 1
    if count == [len(lst) for lst in list_genes][start_row]:
        start_row += 1
        count = 0
labels_extended.head()

,index,genes,sex,origin,price,birth,url,Spotnose,Sugar,KRG,...,Bamboo,Bongo,Desert Ghost,Het Citrus Hypo,Axanthic (TSK),Hypo,Axanthic (VPI),Orange Ghost,Stranger,Het Puzzle
0,0-0,"['Yellow Belly', 'Pastel', 'Het Puzzle', '50% ...",female,Self Produced,650.0,7th April 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1-0,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1-1,"['Piebald', 'Albino']",female,Self Produced,450.0,15th October 2023,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2-0,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2-1,"['Butter', 'Yellow Belly', 'Hurricane', 'Leopa...",male,Self Produced,750.0,1st May 2022,https://www.morphmarket.com/us/c/reptiles/pyth...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
assert list(labels_extended.select_dtypes('int').sum(axis=1)) == [len(lst) for lst in list_genes]

## Load Data